In [ ]:
import re
import collections
import math
import xlrd
import xlsxwriter
import numpy as np
import datetime
import codecs
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.spatial.distance import cdist

In [ ]:
def add_months(s, p):
    ''' 在日期s上加上p个月，如果日期不存在，顺延到下个月1号 '''
    p_years = s.year + p / 12
    p_months = s.month + p % 12 
    if p_months > 12:
        p_years += 1
        p_months = p_months % 12
    
    try:
        rs = datetime.datetime(p_years, p_months, s.day)
    except ValueError, e:
        if p_months == 12:
            rs = datetime.datetime(p_years + 1, 1, 1)
        else:
            rs = datetime.datetime(p_years, p_months+1, 1)
    return rs

def next_quater(cur_quater):
    return add_months(cur_quater, 3)

class SharesInfo(object):
    def __init__(self, shares, marketValue, proportion):
        self.shares_ = shares
        self.marketValue_ = marketValue
        self.proportion_ = proportion
    
    def __str__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)
    def __repr__(self):
        return "%f %f %f"%(self.shares_, self.marketValue_, self.proportion_)


# 季度的分割日期，[datetime.datetime(2004, 1, 1, 0, 0), datetime.datetime(2004, 4, 1, 0, 0), datetime.datetime(2004, 7, 1, 0, 0) 。。。
quater_date = [datetime.datetime(2003,7,1),datetime.datetime(2003,10,1)] \
        + [datetime.datetime(v1,v2,1) for v1 in xrange(2004, 2018) for v2 in xrange(1,12,3)]

In [ ]:
g_raw_data = collections.defaultdict(lambda : collections.defaultdict(dict))
g_record = 0
g_stock_set = set()
def load_data(file_name="Fund_Portfolio_Stock2003-2007.txt"):
    global g_record
    with codecs.open(file_name, "r", encoding="utf_16") as ifid:
        line_num = 0
        for line in ifid:
            if line_num > 0 and len(line) > 5:
                try:
                    MasterFundCode, ReportTypeID, Startdate, EndDate, Rank, Symbol, StockName, Shares, MarketValue, Proportion \
                        = line.strip().split('\t')
                    Startdate = datetime.datetime.strptime(Startdate, "%Y-%m-%d")
                    EndDate = datetime.datetime.strptime(EndDate, "%Y-%m-%d")
                    if int(ReportTypeID) in (1,2,3,4) and Startdate >= quater_date[0] and EndDate < quater_date[-1]:
                        real_start_date = datetime.datetime(Startdate.year, 3*int(ReportTypeID)-2, 1)
                        real_idx = quater_date.index(real_start_date)
                        
                        if quater_date[real_idx] <= Startdate < quater_date[real_idx+1] \
                            and quater_date[real_idx] < EndDate < quater_date[real_idx+1] \
                            and Startdate <= EndDate:
                            g_raw_data[MasterFundCode][Symbol][real_start_date] = SharesInfo(float(Shares), float(MarketValue), float(Proportion))
                            g_stock_set.add(Symbol)
                            g_record += 1
                        else:
                            print u"error at %s linenum %d:%s %s %s"  %(file_name, line_num+1, MasterFundCode, Symbol, Startdate)
                except ValueError, e:
                    print line.strip(), file_name, line_num+1
            line_num += 1


                            
load_data("Fund_Portfolio_Stock2003-2007.txt")
load_data("Fund_Portfolio_Stock2008-2012.txt")
load_data("Fund_Portfolio_Stock2013-2017.txt")
print "加载%d只基金%d条数据，共%d只股票" %(len(g_raw_data), g_record, len(g_stock_set))


In [ ]:
g_holding_num_data = collections.defaultdict(lambda : collections.defaultdict(lambda : np.zeros(len(g_stock_set))))
g_stock_map = {sym_: i_ for i_, sym_ in enumerate(sorted(g_stock_set))}
for fund_ in g_raw_data:
    for sym_ in g_raw_data[fund_]:
        for cur_ in g_raw_data[fund_][sym_]:
            g_holding_num_data[fund_][cur_][g_stock_map[sym_]] = g_raw_data[fund_][sym_][cur_].shares_


In [ ]:
g_trade_vector = collections.defaultdict(dict)
for fund_ in g_holding_num_data:
    start_date_ = sorted(g_holding_num_data[fund_].keys())[0]
    end_date_ = sorted(g_holding_num_data[fund_].keys())[-1]
    for i in xrange(quater_date.index(start_date_)+1, quater_date.index(end_date_)+1):
        if quater_date[i] in g_holding_num_data[fund_]:
            if quater_date[i-1] in g_holding_num_data[fund_]:
                g_trade_vector[quater_date[i]][fund_] = \
                    g_holding_num_data[fund_][quater_date[i]] - g_holding_num_data[fund_][quater_date[i-1]]
            else:
                g_trade_vector[quater_date[i]][fund_] = g_holding_num_data[fund_][quater_date[i]]
        else:
            print u"error at %s %s"%(fund_, quater_date[i])

In [ ]:
g_rank_raw_data = collections.defaultdict(dict)
g_fund_set = set()
with xlrd.open_workbook(u'pref_order_result.xlsx') as wb:
    ws = wb.sheet_by_index(1)
    for r in xrange(1, ws.nrows):
        fund_id = ws.cell(r, 1).value
        quarter = datetime.datetime(int(ws.cell(r,2).value[:4]), int(ws.cell(r,2).value[5])*3-2, 1)
        order = float(ws.cell(r,3).value)
        
        if quarter in quater_date:
            g_rank_raw_data[quarter][fund_id] = order
            g_fund_set.add(fund_id)
        else:
            print "error at %s %s %f"%(fund_id, quarter, order)
            
g_sorted_fund = sorted(g_fund_set)
g_fund_map = {fund: i for i, fund in enumerate(g_sorted_fund)}
g_rank = collections.defaultdict(lambda : np.empty(len(g_sorted_fund), dtype='int64'))
tmp_order = collections.defaultdict(lambda : np.full(len(g_sorted_fund), -1.0))

for quarter in g_rank_raw_data:
    for fund in g_rank_raw_data[quarter]:
        tmp_order[quarter][g_fund_map[fund]] = g_rank_raw_data[quarter][fund]
        
for quarter in tmp_order:
    sorted_idx = np.argsort(tmp_order[quarter])[::-1]
    g_rank[quarter][sorted_idx] = np.arange(len(g_sorted_fund))

In [ ]:
print g_rank[datetime.datetime(2003,7,1)][g_fund_map[u'080001']]
print g_rank_raw_data[datetime.datetime(2003,7,1)][u'080001']

In [ ]:
def calc_sim(trade_vec):
    leading_sim_ = collections.defaultdict(dict)
    current_sim_ = collections.defaultdict(dict)
    following_sim_ = collections.defaultdict(dict)
    following_advance_sim_ = collections.defaultdict(dict)
    
    start_idx_ = quater_date.index(datetime.datetime(2004,1,1))
    end_idx_ = quater_date.index(datetime.datetime(2017,10,1))
    
    for i in xrange(start_idx_, end_idx_):
        current_date = quater_date[i]
        print "calc %s"%(current_date)
        # 去除变化为0的funds，并且排序
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] if trade_vec[current_date][funds_].sum()!=0])
        current_matrix = np.matrix([trade_vec[current_date][funds_] for funds_ in sorted_current_funds])
        current_cos_dis = cdist(current_matrix, current_matrix, 'cosine')        
        
        for i_, fund_ in enumerate(sorted_current_funds):
            current_sim_[fund_][current_date] = (current_cos_dis[i_].sum()-current_cos_dis[i_][i_], len(sorted_current_funds) - 1)
    
    
    cross_start_idx = quater_date.index(datetime.datetime(2004,1,1))
    cross_end_idx = quater_date.index(datetime.datetime(2017,10,1))
    for i in xrange(cross_start_idx, cross_end_idx):
        previous_date = quater_date[i-1]
        current_date= quater_date[i]
        
        sorted_previous_funds = sorted([funds_ for funds_ in trade_vec[previous_date] if trade_vec[previous_date][funds_].sum()!=0])
        sorted_current_funds = sorted([funds_ for funds_ in trade_vec[current_date] if trade_vec[current_date][funds_].sum()!=0])
        
        previous_matrix = np.matrix([trade_vec[previous_date][funds_] for funds_ in sorted_previous_funds])
        current_matrix = np.matrix([trade_vec[current_date][funds_] for funds_ in sorted_current_funds])
        
        cross_cos_dis = cdist(previous_matrix, current_matrix, "cosine")
        
        print "calc cross %s %s, %s %s, %s"%(previous_date, current_date, previous_matrix.shape, current_matrix.shape, cross_cos_dis.shape)
        
        for i_, fund_ in enumerate(sorted_previous_funds):
            if fund_ in sorted_current_funds:
                remove_idx_ = sorted_current_funds.index(fund_) # 减去和T中自己的cos距离
                leading_sim_[fund_][previous_date] = (cross_cos_dis[i_].sum()-cross_cos_dis[i_][remove_idx_], len(sorted_current_funds) - 1)
            else:
                leading_sim_[fund_][previous_date] = (cross_cos_dis[i_].sum(), len(sorted_current_funds))
            
        for i_, fund_ in enumerate(sorted_current_funds):
            if fund_ in sorted_previous_funds:
                remove_idx_ =sorted_previous_funds.index(fund_)
                following_sim_[fund_][current_date] = (cross_cos_dis.T[i_].sum()-cross_cos_dis.T[i_][remove_idx_], len(sorted_previous_funds)-1)
            else:
                following_sim_[fund_][current_date] = (cross_cos_dis.T[i_].sum(), len(sorted_previous_funds))
                
        for i_, fund_ in enumerate(sorted_current_funds):
            order_ = g_rank_raw_data[previous_date].get(fund_, -1.0)
            sorted_previous_advance_flag = [True if g_rank_raw_data[previous_date].get(foo_fund, -2.0) > order_ else False \
                                         for foo_fund in sorted_previous_funds]
            if any(sorted_previous_advance_flag):
                following_advance_sim_[fund_][current_date] = (cross_cos_dis.T[i_][sorted_previous_advance_flag].sum(), \
                                                          sum(sorted_previous_advance_flag))
            
            if fund_ == u'519011' and current_date == datetime.datetime(2004,4,1):
                print order_, sum(sorted_previous_advance_flag)

                
    return leading_sim_, current_sim_, following_sim_, following_advance_sim_

g_leading_sim, g_current_sim, g_following_sim, g_following_advance_sim = calc_sim(g_trade_vector)

In [ ]:
g_jingli_leading_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_current_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_sim = collections.defaultdict(lambda :collections.defaultdict(dict))
g_jingli_following_advance_sim = collections.defaultdict(lambda :collections.defaultdict(dict))

g_jingli_leading_sim_counter = 0
g_jingli_current_sim_counter = 0
g_jingli_following_sim_counter = 0
g_jingli_following_advance_sim_counter = 0

jingli_info = collections.defaultdict(lambda : collections.defaultdict(lambda :["",0]))

with xlrd.open_workbook(u"基金经理.xls") as wb:
    ws = wb.sheet_by_index(0)
    for r in xrange(ws.nrows):
        funds = ws.cell(r,0).value[:6]
        quater = datetime.datetime(int(ws.cell(r,4).value[:4]), 3*int(ws.cell(r,4).value[6])-2, 1)
        name = ws.cell(r,5).value
        num = int(ws.cell(r,6).value)
        if quater not in quater_date:
            print u"error at %s %s"%(funds, quater)
        elif num > jingli_info[funds][quater][1]:            
            jingli_info[funds][quater][0] = name
            jingli_info[funds][quater][1] = num
            
for funds in jingli_info:
    sorted_quater = sorted(jingli_info[funds].keys())
    for idx, quater in enumerate(sorted_quater):
        if idx == 0 or jingli_info[funds][sorted_quater[idx-1]][0] != jingli_info[funds][quater][0]:
            if funds in g_leading_sim and quater in g_leading_sim[funds]:                
                g_jingli_leading_sim_counter += 1
            if funds in g_current_sim and quater in g_current_sim[funds]:                    
                g_jingli_current_sim_counter += 1
            if funds in g_following_sim and quater in g_following_sim[funds]:                
                g_jingli_following_sim_counter += 1
            if funds in g_following_advance_sim and quater in g_following_advance_sim[funds]:                
                g_jingli_following_advance_sim_counter += 1
        
        if funds in g_leading_sim and quater in g_leading_sim[funds]:   
            g_jingli_leading_sim[g_jingli_leading_sim_counter][funds][quater] = g_leading_sim[funds][quater]
        if funds in g_current_sim and quater in g_current_sim[funds]:
            g_jingli_current_sim[g_jingli_current_sim_counter][funds][quater] = g_current_sim[funds][quater]
        if funds in g_following_sim and quater in g_following_sim[funds]:  
            g_jingli_following_sim[g_jingli_following_sim_counter][funds][quater] = g_following_sim[funds][quater]
        if funds in g_following_advance_sim and quater in g_following_advance_sim[funds]:  
            g_jingli_following_advance_sim[g_jingli_following_advance_sim_counter][funds][quater] \
                = g_following_advance_sim[funds][quater]

In [ ]:
a=u"2016年第1季度"
print jingli_info[u'000001'][datetime.datetime(2004,7,1)][0], jingli_info[u'000001'][datetime.datetime(2004,7,1)][1]
for seg_idx in g_jingli_following_sim:
    for funds in g_jingli_following_sim[seg_idx]:
        if funds == u'000001':
            sorted_quarter = sorted(g_jingli_following_sim[seg_idx][funds].keys())
            print seg_idx, funds, sorted_quarter[0], sorted_quarter[-1], len(sorted_quarter)

In [ ]:
workbook = xlsxwriter.Workbook(u'quarter_cosine_distance_jingli.xlsx')
current_ws = workbook.add_worksheet(u"current")
leading_ws = workbook.add_worksheet(u"leading")
following_ws = workbook.add_worksheet(u"following")
following_advance_ws = workbook.add_worksheet(u"following_advance")

def dump_result(ws, jingli_data):
    ws.write(0,0,u"fund")
    ws.write(0,1,u"quater")
    ws.write(0,2,u"sum_distance")
    ws.write(0,3,u"num")
    ws.write(0,4,u"avg_distance")
    ws.write(0,5,u"1-avg")
    ws.write(0,6,u"avg-4(include)")
    ws.write(0,7,u"avg-2(exclude)")
    ws.write(0,8,u"avg-4(exclude)")
    ws.write(0,9,u"avg-6(exclude)")
    
    wt_idx=1
    for seg, data in jingli_data.iteritems():
        for fund_ in data:
            sorted_quater = sorted(data[fund_].keys(), reverse=True)
            for i, quater_ in enumerate(sorted_quater):
                ws.write(wt_idx,0,fund_)
                ws.write(wt_idx,1,u"%dQ%d"%(quater_.year, quater_.month/3+1))
                ws.write(wt_idx,2,data[fund_][quater_][0])
                ws.write(wt_idx,3,data[fund_][quater_][1])
                ws.write(wt_idx,4,data[fund_][quater_][0]/data[fund_][quater_][1])
                ws.write(wt_idx,5,1-data[fund_][quater_][0]/data[fund_][quater_][1])

                total = 0.0
                end_idx = min((i+4, len(sorted_quater)))
                for j in xrange(i, end_idx):
                    total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                ws.write(wt_idx,6,total/(end_idx-i))

                if i < len(sorted_quater)-1:
                    total = 0.0
                    end_idx = min((i+3, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,7,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+5, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,8,total/(end_idx-i-1))

                    total = 0.0
                    end_idx = min((i+7, len(sorted_quater)))
                    for j in xrange(i+1, end_idx):
                        total += (1-data[fund_][sorted_quater[j]][0]/data[fund_][sorted_quater[j]][1])
                    ws.write(wt_idx,9,total/(end_idx-i-1))
                else:
                    ws.write(wt_idx,7,u'.')
                    ws.write(wt_idx,8,u'.')
                    ws.write(wt_idx,9,u'.')

                wt_idx += 1 
            
dump_result(current_ws, g_jingli_current_sim)
dump_result(leading_ws, g_jingli_leading_sim)
dump_result(following_ws, g_jingli_following_sim)
dump_result(following_advance_ws, g_jingli_following_advance_sim)


workbook.close()

In [ ]:
print g_following_advance_sim[u'110002'][datetime.datetime(2004,10,1)]